In [217]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory,StopWordRemover,ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory 
import numpy as np #Operasi Matematika dan linear aljebra 
import pandas as pd #data processing
import matplotlib.pyplot as plt #Visualisasi data 
%matplotlib inline
import seaborn as sns #Visualisasi data import string
import nltk
# nltk.download("punkt")
from nltk.tokenize import RegexpTokenizer 
# from nltk.corpus import stopwords nltk.download('stopwords')
from nltk.stem import PorterStemmer
from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.feature_extraction.text import TfidfVectorizer 
import logging
import pickle
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer 
import time
from scipy import spatial
import math
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV 
import heapq
from sklearn.neighbors import KNeighborsClassifier 
from sklearn import metrics
import re
import requests
import torch
from random import randrange
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [142]:
# !jupyter nbconvert --to script dupakq.ipynb

[NbConvertApp] Converting notebook dupakq.ipynb to script
[NbConvertApp] Writing 3894 bytes to dupakq.py


In [7]:
pd.set_option("display.max_colwidth",None)

# load data

## butir penunjang

In [349]:
df_penunjang=pd.read_excel("data/butir_penunjang.xlsx")#,sep="\t",engine="python") df21=df21[df21.IP.notna()]
df_penunjang.reset_index(inplace=True,drop=True)
cols=list(df_penunjang.columns)


def replace_emptystring_with_na(row):
    for idx,val in row.items():
        try:
            txt=str(val).replace(u'\xa0',"xa0")#.encode('utf-8')
            if txt=="xa0":
                row[idx]=np.NaN    
        except Exception as e:
            print("{}:{}-{}".format(idx,val,str(e)))
        
    return row
df_penunjang.apply(replace_emptystring_with_na,axis=1)

df_penunjang.fillna(method="ffill",inplace=True)


penunjang_activity_cols = ['kegiatan', 'Unnamed: 5','Unnamed: 6'] #['unsur', 'sub_unsur', 'kegiatan', 'Unnamed: 5','Unnamed: 6']
penunjang_activity_code_cols = ['nomor_unsur', 'nomor_sub_unsur', 'kegiatan', 'Unnamed: 5','Unnamed: 6']
def merge_activities(row):
    arr=[]
    for k,v in row.items():
        try:
            if len(str(v))>3:
                arr.append(str(v))
        except:
            continue
    return ".".join(arr)

def merge_activity_codes(row):
    arr=[]
    for k,v in row.items():
#         print(v)
        if (not pd.isnull(v)) and (not pd.isna(v)) and (str(v).lower()!="nan"):
            try:
                if len(str(v))<=3:
                    arr.append(str(v))
            except:
                print("ERROR on key {}".format(k))
                continue
        else:
            print("NAN on {}:{}".format(k,v))
    res=".".join(arr)
    if "nan" in res:
        print("row idx: {}".format(row.index))
    return res

# df_penunjang.drop("activity_code",axis=1,inplace=True)
df_penunjang["activities"]=df_penunjang[penunjang_activity_cols].apply(merge_activities,axis=1)
df_penunjang["activity_code"]=df_penunjang[penunjang_activity_code_cols].apply(merge_activity_codes,axis=1)

df_penunjang_ori=pd.read_excel("data/butir_penunjang.xlsx")#,sep="\t",engine="python") df21=df21[df21.IP.notna()]
for idx,row in df_penunjang_ori.iterrows():
    df_penunjang.at[idx,"ak"]=df_penunjang_ori.at[idx,"ak"]
    if not re.findall("\d+",str(row.ak)):
#         df_penunjang.loc[idx]=df_penunjang_ori.loc[idx]
        df_penunjang.at[idx,"ak"]=np.NaN
    
df_penunjang.to_csv("data/butir_penunjang.csv",sep=";")



NAN on Unnamed: 5:nan
NAN on Unnamed: 6:nan
NAN on Unnamed: 6:nan


# sanity test

In [350]:
df_penunjang["activity_code"].unique()

array(['IV.A', 'IV.B.1', 'IV.B.1.a', 'IV.B.1.b', 'IV.B.1.c', 'IV.B.2',
       'IV.B.2.a', 'IV.B.2.b', 'IV.B.3', 'IV.B.3.a', 'IV.B.3.b', 'IV.B.4',
       'IV.B.4.a', 'IV.B.4.b', 'IV.B.5', 'IV.B.6', 'IV.C.1', 'IV.C.1.a',
       'IV.C.1.b', 'IV.C.2', 'IV.C.2.a', 'IV.C.2.b', 'IV.D.b', 'IV.E.b',
       'IV.E.1', 'IV.E.2', 'IV.E.3', 'IV.E.3.a', 'IV.E.3.b', 'IV.E.3.c',
       'IV.E.3.d', 'IV.E.3.e', 'IV.E.3.f', 'IV.E.3.g', 'IV.E.4',
       'IV.E.4.a', 'IV.E.4.b', 'IV.E.4.c', 'IV.E.4.d', 'IV.E.4.e',
       'IV.E.4.f', 'IV.E.4.g', 'IV.E.5', 'IV.F', 'IV.F.a', 'IV.F.b',
       'IV.F.c', 'IV.F.d', 'IV.F.e', 'IV.F.f', 'IV.F.g', 'IV.F.h', 'V.A',
       'V.B', 'V.C.1', 'V.C.1.a', 'V.C.1.b', 'V.C.1.c', 'V.C.2',
       'V.C.2.a', 'V.C.2.b', 'V.C.2.c', 'V.D.c', 'V.D.a', 'V.D.b',
       'V.D.d', 'V.E', 'V.E.a', 'V.E.b'], dtype=object)

In [351]:
len(df_penunjang.activity_code.unique()),len(df_penunjang.activity_code)

(69, 70)

In [352]:
df_penunjang[df_penunjang.activity_code.duplicated()][penunjang_activity_code_cols+["activity_code"]].head(3)

,nomor_unsur,nomor_sub_unsur,kegiatan,Unnamed: 5,Unnamed: 6,activity_code
65,V,D,c,Magister,Tingkat Provinsi,V.D.c


In [353]:
df_penunjang[df_penunjang.activity_code=="V.D.c"]

,nomor_unsur,unsur,nomor_sub_unsur,sub_unsur,kegiatan,Unnamed: 5,Unnamed: 6,ak,jenjang,satuan_hasil,bukti_fisik,batas_penilaian,keterangan_hasil_review,activities,activity_code
62,V,Penunjang Kegiatan teknologi informasi berbasis komputer,D,Perolehan Gelar Kesarjanaan\nLainnya,Memperoleh gelar kesarjanaan lainnya yang tidak sesuai dengan bidang tugas Pranata Komputer,c,Tingkat Provinsi,NaN,Semua Jenjang,Sertifikat/Piagam,Fotokopi sertifikat/piagam/surat keterangan,Tidak ada pada juknis,3 kali setahun,Memperoleh gelar kesarjanaan lainnya yang tidak sesuai dengan bidang tugas Pranata Komputer.Tingkat Provinsi,V.D.c
65,V,Penunjang Kegiatan teknologi informasi berbasis komputer,D,Perolehan Gelar Kesarjanaan\nLainnya,c,Magister,Tingkat Provinsi,10,Semua Jenjang,Ijazah,Fotocopy ijazah kesarjanaan dan transkrip yang telah\ndilegalisir instansi yang berwenang sesuai dengan ketentuan\nyang berlaku serta surat tugas/ijin belajar,Tidak ada pada juknis,3 kali setahun,Magister.Tingkat Provinsi,V.D.c


In [354]:
df_penunjang=df_penunjang[df_penunjang.ak.notna()]

In [355]:
len(df_penunjang.activity_code.unique()),len(df_penunjang.activity_code)

(56, 56)

In [266]:
activity_code_cols = ['nomor_unsur', 'nomor_sub_unsur', 'kegiatan', 'Unnamed: 5','Unnamed: 6']
df_penunjang[df_penunjang["activity_code"].str.len()==3][activity_code_cols+["activity_code"]]

,nomor_unsur,nomor_sub_unsur,kegiatan,Unnamed: 5,Unnamed: 6,activity_code
52,V,A,Mengajar/melatih/membimbing yang berkaitan dengan bidang teknologi informasi berbasis komputer,Melakukan Kegiatan Pembinaan Jabatan Fungsional Pranata Komputer,Lamanya kurang dari 30 jam,V.A
53,V,B,Menjadi anggota Tim Penilai/Tim Uji Kompetensi,Melakukan Kegiatan Pembinaan Jabatan Fungsional Pranata Komputer,Lamanya kurang dari 30 jam,V.B


In [23]:
df_penunjang.loc[2,["activity_code","activities","ak"]]

activity_code                                                                                       IV.B.1.a
activities       dalam bentuk buku/majalah ilmiah internasional yang diterbitkan internasional yang terindek
ak                                                                                                        20
Name: 2, dtype: object

In [267]:
df_penunjang[df_penunjang.ak.notna()][["activity_code","activities","ak"]].head()

,activity_code,activities,ak
0,IV.A,Memperoleh ijazah sesuai dengan bidang tugas Jabatan Fungsional Pranata Komputer,25% AK kenaikan\npangkat
2,IV.B.1.a,dalam bentuk buku/majalah ilmiah internasional yang diterbitkan internasional yang terindek,20
3,IV.B.1.b,dalam bentuk buku/majalah ilmiah internasional yang diterbitkan nasional,12.5
4,IV.B.1.c,dalam bentuk buku/majalah ilmiah internasional yang diterbitkan dan diakui oleh organisasi profesi dan Instansi Pembina,6
6,IV.B.2.a,dalam bentuk buku,8


# prepare butir_trampil

In [285]:
df_trampil=pd.read_excel("data/butir_tenaga_trampil.xlsx")#,sep="\t",engine="python") df21=df21[df21.IP.notna()]
cols=list(df_trampil.columns)

def replace_emptystring_with_na(row):
    for idx,val in row.items():
        try:
            txt=str(val).replace(u'\xa0',"xa0")#.encode('utf-8')
            if txt=="xa0":
                row[idx]=np.NaN    
        except Exception as e:
            print("{}:{}-{}".format(idx,val,str(e)))
        
    return row

df_trampil.apply(replace_emptystring_with_na,axis=1)
df_trampil.fillna(method="ffill",inplace=True)


activity_cols = ['kegiatan'] #['unsur', 'sub_unsur', 'kegiatan', 'Unnamed: 4']#,'Unnamed: 6']
activity_code_cols = ['nomor_unsur', 'nomor_sub_unsur', 'kegiatan', 'Unnamed: 4']#,'Unnamed: 6']
def merge_activities(row):
    arr=[]
    for k,v in row.items():
        try:
            if len(str(v))>3:
                arr.append(str(v))
        except:
            continue
    return ".".join(arr)

def merge_activity_codes(row):
    arr=[]
    for k,v in row.items():
        try:
            if len(str(v))<=3:
                arr.append(str(v))
        except:
            continue
    return ".".join(arr)

df_trampil["activities"]=df_trampil[activity_cols].apply(merge_activities,axis=1)
df_trampil["activity_code"]=df_trampil[activity_code_cols].apply(merge_activity_codes,axis=1)

df_trampil_ori=pd.read_excel("data/butir_tenaga_trampil.xlsx")#,sep="\t",engine="python") df21=df21[df21.IP.notna()]
for idx,row in df_trampil_ori.iterrows():
    df_trampil.at[idx,"ak"]=df_trampil_ori.at[idx,"ak"]
    if not re.findall("\d+",str(row.ak)):
        df_trampil.at[idx,"ak"]=np.NaN
    
df_trampil.to_csv("data/butir_trampil.csv",sep=";")

In [286]:
df_trampil["activity_code"].unique()

array(['I.A.1', 'I.B.1', 'I.B.2', 'I.B.3', 'I.B.4', 'I.B.5', 'I.B.6',
       'I.B.7', 'I.B.8', 'I.B.9', 'I.B.10', 'I.B.11', 'I.B.12', 'I.B.13',
       'I.B.14', 'I.B.15', 'I.B.16', 'I.B.17', 'I.B.18', 'I.B.19',
       'I.B.20', 'I.B.21', 'I.B.22', 'I.B.23', 'I.B.24', 'I.B.25',
       'I.B.26', 'I.B.27', 'I.C.1', 'I.C.2', 'II.A.1', 'II.A.2', 'II.A.3',
       'II.A.4', 'II.A.5', 'II.A.6', 'II.A.7', 'II.A.8', 'II.A.9',
       'II.A.10', 'II.B.1', 'II.B.2', 'II.B.3', 'II.B.4', 'II.B.5',
       'II.B.6', 'II.B.7', 'II.B.8', 'II.B.9', 'II.B.10', 'III.A.1',
       'III.A.2', 'III.A.3', 'III.A.4', 'III.A.5', 'III.A.6', 'III.B.1',
       'III.B.2', 'III.B.3', 'III.B.4', 'III.B.5', 'III.B.6', 'III.B.7',
       'III.C.1', 'III.C.2', 'III.C.3', 'III.C.4', 'III.C.5', 'III.C.6',
       'III.C.7', 'III.C.8', 'III.C.9', 'III.C.10', 'III.C.11'],
      dtype=object)

In [268]:
activity_code_cols = ['nomor_unsur', 'nomor_sub_unsur', 'kegiatan', 'Unnamed: 4']
df_trampil[df_trampil["activity_code"].str.len()==3][activity_code_cols+["activity_code"]]

,nomor_unsur,nomor_sub_unsur,kegiatan,Unnamed: 4,activity_code
50,III,A,Menyusun petunjuk operasional program aplikasi,1,A.1
51,III,A,Menyusun dokumentasi pengembangan sistem informasi,2,A.2
52,III,A,Melakukan analisis kebutuhan program aplikasi,3,A.3
53,III,A,Membuat program aplikasi,4,A.4
54,III,A,Mengembangkan dan atau meremajakan program aplikasi,5,A.5
55,III,A,Melakukan uji coba program aplikasi,6,A.6
56,III,B,Melakukan perekaman data dengan pemindaian,1,B.1
57,III,B,Melakukan perekaman data tanpa validasi,2,B.2
58,III,B,Melakukan validasi hasil perekaman data,3,B.3
59,III,B,Melakukan perekaman data dengan validasi,4,B.4


In [11]:
df_trampil[df_trampil.ak.notna()][["activity_code","activities","ak"]].head()

,activity_code,activities,ak
0,I.A.1,Melakukan pemenuhan permintaan dan layanan teknologi informasi,0.15
1,I.B.1,Melakukan pengumpulan informasi mengenai data instansi,0.55
2,I.B.2,Melakukan pengadministrasian kegiatan tata kelola data,0.11
3,I.B.3,Melakukan pencatatan permasalahan pengelolaan data,0.11
4,I.B.4,Melakukan perawatan arsitektur teknologi data,0.055


# prepare butir_ahli

In [287]:
df_ahli=pd.read_excel("data/butir_tenaga_ahli.xlsx")#,sep="\t",engine="python") df21=df21[df21.IP.notna()]
cols=list(df_ahli.columns)

def replace_emptystring_with_na(row):
    for idx,val in row.items():
        try:
            txt=str(val).replace(u'\xa0',"xa0")#.encode('utf-8')
            if txt=="xa0":
                row[idx]=np.NaN    
        except Exception as e:
            print("{}:{}-{}".format(idx,val,str(e)))
        
    return row

df_ahli.apply(replace_emptystring_with_na,axis=1)
df_ahli.fillna(method="ffill",inplace=True)


activity_cols = ['kegiatan'] #['unsur', 'sub_unsur', 'kegiatan', 'Unnamed: 4']#,'Unnamed: 6']
activity_code_cols = ['nomor_unsur', 'nomor_sub_unsur', 'kegiatan', 'Unnamed: 4']#,'Unnamed: 6']
def merge_activities(row):
    arr=[]
    for k,v in row.items():
        try:
            if len(str(v))>3:
                arr.append(str(v))
        except:
            continue
    return ".".join(arr)

def merge_activity_codes(row):
    arr=[]
    for k,v in row.items():
        try:
            if len(str(v))<=3:
                arr.append(str(v))
        except:
            continue
    return ".".join(arr)

df_ahli["activities"]=df_ahli[activity_cols].apply(merge_activities,axis=1)
df_ahli["activity_code"]=df_ahli[activity_code_cols].apply(merge_activity_codes,axis=1)

df_ahli_ori=pd.read_excel("data/butir_tenaga_ahli.xlsx")#,sep="\t",engine="python") df21=df21[df21.IP.notna()]
for idx,row in df_ahli_ori.iterrows():
    df_ahli.at[idx,"ak"]=df_ahli_ori.at[idx,"ak"]
    if not re.findall("\d+",str(row.ak)):
        df_ahli.at[idx,"ak"]=np.NaN
    
df_ahli.to_csv("data/butir_ahli.csv",sep=";")

In [288]:
df_ahli.activity_code.unique()

array(['I.A.1', 'I.A.2', 'I.A.3', 'I.A.4', 'I.A.5', 'I.A.6', 'I.A.7',
       'I.A.8', 'I.A.9', 'I.A.10', 'I.A.11', 'I.A.12', 'I.A.13', 'I.A.14',
       'I.A.15', 'I.A.16', 'I.A.17', 'I.A.18', 'I.A.19', 'I.A.20',
       'I.A.21', 'I.A.22', 'I.A.23', 'I.A.24', 'I.A.25', 'I.A.26',
       'I.A.27', 'I.A.28', 'I.A.29', 'I.A.30', 'I.A.31', 'I.A.32',
       'I.A.33', 'I.A.34', 'I.A.35', 'I.A.36', 'I.A.37', 'I.B.1', 'I.B.2',
       'I.B.3', 'I.B.4', 'I.B.5', 'I.B.6', 'I.B.7', 'I.B.8', 'I.B.9',
       'I.B.10', 'I.B.11', 'I.B.12', 'I.B.13', 'I.B.14', 'I.B.15',
       'I.B.16', 'I.B.17', 'I.B.18', 'I.B.19', 'I.B.20', 'I.B.21',
       'I.C.1', 'I.C.2', 'I.C.3', 'I.C.4', 'I.C.5', 'I.C.6', 'I.C.7',
       'I.C.8', 'I.C.9', 'I.C.10', 'I.C.11', 'I.C.12', 'I.C.13', 'I.C.14',
       'I.C.15', 'I.C.16', 'I.C.17', 'I.C.18', 'I.C.19', 'I.C.20',
       'I.C.21', 'I.C.22', 'I.C.23', 'I.C.24', 'I.C.25', 'I.C.26',
       'I.C.27', 'I.C.28', 'I.C.29', 'I.C.30', 'I.C.31', 'I.C.32',
       'I.C.33', 'I.C.34', 'I

In [13]:
df_ahli[df_ahli.ak.notna()][["activity_code","activities","ak"]].head()

,activity_code,activities,ak
0,I.A.1,Melakukan Identifikasi dan Analisis Kebutuhan Bisnis Institusi,4.16
1,I.A.2,Melakukan Reviu TI Yang Digunakan Institusi Saat Ini Atau Tren TI Terkinii,0.78
2,I.A.3,Melakukan Analisis Terhadap Strategi Bisnis Institusi yang Berdampak Pada Strategi TI Institusi,2.20
3,I.A.4,Melakukan Analisis Dampak TI yang Digunakan Institusi Saat Ini dan Trennya Terhadap Perubahan Strategi Bisnis Institusi,1.32
4,I.A.5,Menyusun Kerangka Kerja Untuk Penyusunan Strategi TI,1.32


# check num of activity_code equals to num or records

In [426]:
len(df_penunjang.activity_code.unique()),len(df_penunjang.activity_code)

(56, 56)

In [428]:
len(df_penunjang.activity_code.unique()),len(df_penunjang.activity_code.to_list())

(56, 56)

In [357]:
len(df_trampil.activity_code.unique()),len(df_trampil.activity_code)

(74, 74)

In [358]:
len(df_ahli.activity_code.unique()),len(df_ahli.activity_code)

(206, 206)

# merge 3 dataframes

In [359]:
common_cols=np.intersect1d(df_trampil.columns,df_ahli.columns)
common_cols=np.intersect1d(common_cols,df_penunjang.columns)
common_cols

array(['activities', 'activity_code', 'ak', 'batas_penilaian', 'jenjang',
       'kegiatan', 'nomor_sub_unsur', 'nomor_unsur', 'sub_unsur', 'unsur'],
      dtype=object)

In [360]:
df_trampil2=df_trampil[common_cols]
df_ahli2=df_ahli[common_cols]
df_penunjang2=df_penunjang[common_cols]

In [427]:
frames=[df_trampil2,df_ahli2,df_penunjang2]
df_dupak_all=pd.concat(frames)
df_trampil2.shape,df_ahli2.shape,df_penunjang2.shape,df_dupak_all.shape

((74, 10), (206, 10), (56, 10), (336, 10))

In [429]:
len(df_dupak_all.activity_code.unique()),len(df_dupak_all.activity_code.to_list())

(268, 336)

In [433]:
df_dupak_all[df_dupak_all.activity_code=="I.A.1"]

,activities,activity_code,ak,batas_penilaian,jenjang,kegiatan,nomor_sub_unsur,nomor_unsur,sub_unsur,unsur
0,Melakukan pemenuhan permintaan dan layanan teknologi informasi,I.A.1,0.15,1 (satu) laporan per bulan,Mahir,Melakukan pemenuhan permintaan dan layanan teknologi informasi,A,I,Manajemen Layanan TI,Tata Kelola dan Tata Laksana Teknologi Informasi
0,Melakukan Identifikasi dan Analisis Kebutuhan Bisnis Institusi,I.A.1,4.16,Jumlah maksimum kegiatan identifikasi dan analisis kebutuhan bisnis institusi yang dapat dinilai adalah 2 (dua) kali per tahun.\n,Utama,Melakukan Identifikasi dan Analisis Kebutuhan Bisnis Institusi,A,I,IT Enterprise,Tata Kelola dan Tata Laksana Teknologi Informasi


# add info jenjang into acivity _code

In [437]:
df_dupak_all[df_dupak_all.jenjang.notna()].shape[0]==df_dupak_all.shape[0]

True

In [438]:
df_dupak_all.jenjang.unique()

array(['Mahir', 'Penyelia', 'Terampil', 'Utama', 'Madya', 'Muda',
       'Pertama', 'Semua Jenjang'], dtype=object)

In [439]:
def merge_jenjang_and_code(row):
    jenjang=row.jenjang
    code=row.activity_code
    new_code="_".join([jenjang,code])
    return new_code

df_dupak_all.activity_code=df_dupak_all.apply(merge_jenjang_and_code,axis=1)
df_dupak_all.to_csv("data/dupak_all.csv",sep=";")

In [441]:
df_dupak_all.activity_code.unique().shape[0]==df_dupak_all.activity_code.shape[0]

True

In [442]:
df_dupak_all.activity_code.head(4)

0    Mahir_I.A.1
1    Mahir_I.B.1
2    Mahir_I.B.2
3    Mahir_I.B.3
Name: activity_code, dtype: object

# get tfidf

In [33]:
# nltk.download('stopwords')
# nltk.download('punkt')
# len(activity_cleansed)

In [443]:
df_dupak_all.columns

Index(['activities', 'activity_code', 'ak', 'batas_penilaian', 'jenjang',
       'kegiatan', 'nomor_sub_unsur', 'nomor_unsur', 'sub_unsur', 'unsur'],
      dtype='object')

In [444]:
tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
factory = StemmerFactory()
stemmer = factory.create_stemmer()
# default_stopwords = StopWordRemoverFactory().get_stop_words()
# additional_stopwords=["(",")","senin","selasa","rabu","kamis","jumat","sabtu","minggu"]
# dictionary=ArrayDictionary(default_stopwords+additional_stopwords)
# id_stopword = StopWordRemover(dictionary)
tfidf_model_fp="model/tfdif_vectorizer.pkl"
en_stemmer = PorterStemmer()
en_stopwords = nltk.corpus.stopwords.words('english')

df_id_stopword=pd.read_csv("data/stopwordbahasa.csv",header=None)
id_stopword=df_id_stopword[0].to_list()

def tokenize_clean(text):
    if(text):
        tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word
            in nltk.word_tokenize(sent)]
        #clean token from numeric and other character like puntuation
        filtered_tokens = []
        for token in tokens:
            txt=re.findall('[a-zA-Z]{3,}', token)
            if txt:
                filtered_tokens.append(txt[0])       
        return filtered_tokens

def remove_stopwords(tokenized_text):
    if(tokenized_text):
        cleaned_token = []
        for token in tokenized_text:
            if token not in id_stopword:
                cleaned_token.append(token)

        return cleaned_token

def stem_text(tokenized_text):
    if (tokenized_text):
        stems = []
        for token in tokenized_text:
            stems.append(stemmer.stem(token))

        return stems

def remove_en_stopwords(text):
    if text:
        return [token for token in text if token not in en_stopwords]

def stem_en_text(text):
    if text:
        return [en_stemmer.stem(word) for word in text]

def revome_slash_n(text):
    if text:
        return [str(txt).replace("\n"," ") for txt in text]

def lower_text(text):
    if text:
        return [str(txt).lower() for txt in text]

def make_sentence(arr):
    if arr:
        return " ".join(arr)

def text_preprocessing(text):
    if text:
        prep01 = tokenize_clean(text)
        prep02 = remove_stopwords(prep01)
        prep03 = stem_text(prep02)
        prep04 = remove_en_stopwords(prep03)
        prep05 = stem_en_text(prep04)
        prep06 = revome_slash_n(prep05)
        prep07 = lower_text(prep06)
        prep08 = make_sentence(prep07)
        return prep08

tqdm.pandas()
df_dupak_all=df_dupak_all[df_dupak_all.ak.notna()]
df_dupak_all["activity_cleansed"]=df_dupak_all.activities.progress_apply(text_preprocessing)
activity_cleansed=df_dupak_all.activity_cleansed.to_list()
activity_codes=df_dupak_all.activity_code.to_list()

# tfidf_vectorizer=TfidfVectorizer(use_idf=True) 
# tfidf_vecorizer=TfidfVectorizer(max_df=0.8,             # terms with document frequency value > 0.8 will be removed
#                     min_df=0.2,             # terms with document frequency value < 0.2 will be removed
#                     max_features=200000,    # create maximum 200.000 vocabulary that only consider the top max_features ordered by term frequency across the corpus.
#                     stop_words = stopwords, # stopwords list
#                     use_idf=True,           # enable inverse-document-frequency reweighting
#                     tokenizer=text_preprocessing, # override the string tokenization step by using text_prepocessing function 
#                     ngram_range=(1,1))


100%|██████████| 336/336 [00:23<00:00, 14.57it/s]


# train tfidf

In [448]:
df_dupak_all.activities.head(5),activity_cleansed[:5]

(0    Melakukan pemenuhan permintaan dan layanan teknologi informasi
 1            Melakukan pengumpulan informasi mengenai data instansi
 2            Melakukan pengadministrasian kegiatan tata kelola data
 3                Melakukan pencatatan permasalahan pengelolaan data
 4                     Melakukan perawatan arsitektur teknologi data
 Name: activities, dtype: object,
 ['penuh minta layan teknolog informasi',
  'kumpul informasi data instansi',
  'administrasi giat tata kelola data',
  'catat masalah kelola data',
  'awat arsitektur teknolog data'])

In [449]:
tfidf_vectorizer=TfidfVectorizer(use_idf=True,ngram_range=(1,2))
fitted_tfidf_vectorizer = tfidf_vectorizer.fit(activity_cleansed)
with open(tfidf_model_fp, "wb") as fo:
        pickle.dump(fitted_tfidf_vectorizer, fo)
        print("SAVED tfidf vectorizer to ", tfidf_model_fp)


SAVED tfidf vectorizer to  model/tfdif_vectorizer.pkl


# transform activities to tfidf vector

In [450]:
def vectorize_tfidf(txt):
    densematrix=fitted_tfidf_vectorizer.transform([txt])
    skillvecs=densematrix.toarray().tolist()
    vector=np.array(skillvecs[0]).astype('float32').tolist()
#     vector=np.expand_dims(vector,0).tolist()
    return vector

df_dupak_all["tfidf_vec"]=df_dupak_all.activity_cleansed.progress_apply(vectorize_tfidf)
df_dupak_all.to_csv("data/dupak_all.csv",sep=";")

100%|██████████| 336/336 [00:00<00:00, 626.94it/s]


# train and query using KNN

In [451]:
df_dupak_all.reset_index(inplace=True, drop=True)
df_dupak_all.index

RangeIndex(start=0, stop=336, step=1)

In [452]:
knn_model_path="model/knn.pkl"
knn_index_path="model/knn_idx.pkl"
knn = KNeighborsClassifier()
knn_index={}
activities=df_dupak_all.activities.to_list()
vecs=df_dupak_all.tfidf_vec.to_list()
idxs=df_dupak_all.index.to_list()
for i,val in enumerate(activities):
    knn_index[i]=val
knn.fit(vecs, idxs)
with open(knn_model_path,"wb") as fi:
  pickle.dump(knn,fi)
  print("train KNN DONE. Model saved in ",knn_model_path)
with open(knn_index_path,"wb") as fi:
  pickle.dump(knn_index,fi)
  print("Index saved in ",knn_index_path)

train KNN DONE. Model saved in  model/knn.pkl
Index saved in  model/knn_idx.pkl


# query using cosine similarity

In [453]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

def cosine_similarity_T(k, query):
    preprocessed_query = re.sub("\W+", " ", query).strip()
    q_cleansed=text_preprocessing(preprocessed_query)
    q_vector=vectorize_tfidf(q_cleansed)
    d_cosines = []
    for i,vec in enumerate(df_dupak_all.tfidf_vec.to_list()):
        d_cosines.append(cosine_sim(q_vector, vec))
    print(d_cosines[:5])
                    
    out = np.array(d_cosines).argsort()[-k:][::-1]
    d_cosines.sort()
    a = pd.DataFrame()
    for i,index in enumerate(out):
        a.loc[i,'index'] = str(index)
        a.loc[i,'Subject'] = activities[index]
    for j,simScore in enumerate(d_cosines[-k:][::-1]):
        a.loc[j,'Score'] = simScore
    return a

cosine_similarity_T(10,"karya tulis ilmiah internasional")

[0.0, 0.0, 0.0, 0.0, 0.0]


,index,Subject,Score
0,317,"Menjadi Reviewer Karya Tulis Ilmiah, Jurnal, Konferensi, Forum Ilmiah\ndan Lain-Lain di Bidang Teknologi Informasi.Lamanya kurang dari 30 jam",0.466979
1,318,"Menjadi Editor Karya Tulis Ilmiah, Jurnal, Konferensi, Forum Ilmiah\ndan Lain-Lain di Bidang Teknologi Informasi.Lamanya kurang dari 30 jam",0.466979
2,281,dalam bentuk buku/majalah ilmiah internasional yang diterbitkan internasional yang terindek,0.357586
3,282,dalam bentuk buku/majalah ilmiah internasional yang diterbitkan nasional,0.319940
4,283,dalam bentuk buku/majalah ilmiah internasional yang diterbitkan dan diakui oleh organisasi profesi dan Instansi Pembina,0.241575
5,327,Tingkat Internasional,0.202450
6,290,"Menyampaikan prasaran berupa tinjauan, gagasan dan atau ulasan ilmiah dalam pertemuan ilmiah.dalam bentuk makalah",0.121250
7,293,dalam majalah ilmiah yang diakui oleh organisasi\nprofesi dan Instansi Pembina,0.077758
8,287,dalam majalah ilmiah yang diakui oleh organisasi\nprofesi dan Instansi Pembina,0.077758
9,89,Menyusun strategi implementasi Enterprise Architecture (EA ),0.000000


# compare cosine vs knn searching

In [454]:
%%time
cosine_similarity_T(5,"karya tulis ilmiah internasional")

[0.0, 0.0, 0.0, 0.0, 0.0]
CPU times: user 284 ms, sys: 16.2 ms, total: 300 ms
Wall time: 283 ms


,index,Subject,Score
0,317,"Menjadi Reviewer Karya Tulis Ilmiah, Jurnal, Konferensi, Forum Ilmiah\ndan Lain-Lain di Bidang Teknologi Informasi.Lamanya kurang dari 30 jam",0.466979
1,318,"Menjadi Editor Karya Tulis Ilmiah, Jurnal, Konferensi, Forum Ilmiah\ndan Lain-Lain di Bidang Teknologi Informasi.Lamanya kurang dari 30 jam",0.466979
2,281,dalam bentuk buku/majalah ilmiah internasional yang diterbitkan internasional yang terindek,0.357586
3,282,dalam bentuk buku/majalah ilmiah internasional yang diterbitkan nasional,0.319940
4,283,dalam bentuk buku/majalah ilmiah internasional yang diterbitkan dan diakui oleh organisasi profesi dan Instansi Pembina,0.241575


In [455]:
%%time
q="karya tulis ilmiah internasional"
q_cleansed=text_preprocessing(q)
q_vector=vectorize_tfidf(q_cleansed)
# densematrix = fitted_tfidf_vectorizer.transform([text_preprocessing(q)])
# skillvecs=densematrix.toarray()
# vector=np.array(skillvecs[0]).astype('float32')
# vector=np.expand_dims(vector,0)
(distances, indices) = knn.kneighbors([q_vector],n_neighbors=5)
indices = indices.tolist()
res=[knn_index[x] for x in indices[0]]
print("Q:{}".format(q_cleansed))
for i,x in enumerate(res):
    print("idx:{}, distance:{}-->{}){}".format(indices[0][i],distances[0][i],i,x))

Q:karya tuli ilmiah internasion
idx:317, distance:1.0324931552138348-->0)Menjadi Reviewer Karya Tulis Ilmiah, Jurnal, Konferensi, Forum Ilmiah
dan Lain-Lain di Bidang Teknologi Informasi.Lamanya kurang dari 30 jam
idx:318, distance:1.0324931552138348-->1)Menjadi Editor Karya Tulis Ilmiah, Jurnal, Konferensi, Forum Ilmiah
dan Lain-Lain di Bidang Teknologi Informasi.Lamanya kurang dari 30 jam
idx:281, distance:1.133502204488936-->2)dalam bentuk buku/majalah ilmiah internasional yang diterbitkan internasional yang terindek
idx:282, distance:1.1662417687992928-->3)dalam bentuk buku/majalah ilmiah internasional yang diterbitkan nasional
idx:283, distance:1.2316047194913837-->4)dalam bentuk buku/majalah ilmiah internasional yang diterbitkan dan diakui oleh organisasi profesi dan Instansi Pembina
CPU times: user 15.1 ms, sys: 3.46 ms, total: 18.5 ms
Wall time: 7.8 ms


# test prediction feature

In [456]:
%%time
q2="membuat karya tulis ilmiah internasional"
q_cleansed2=text_preprocessing(q2)
q_vector2=vectorize_tfidf(q_cleansed2)
predct_idx=knn.predict([q_vector2])
knn_index[list(predct_idx)[0]]

CPU times: user 17.5 ms, sys: 4.28 ms, total: 21.8 ms
Wall time: 8.48 ms


'dalam bentuk buku/majalah ilmiah internasional yang diterbitkan internasional yang terindek'

In [17]:
# !jupyter nbextension install --user https://rawgithub.com/minrk/ipython_extensions/master/nbextensions/toc.js
# !curl -L https://rawgithub.com/minrk/ipython_extensions/master/nbextensions/toc.css > $(jupyter --data-dir)/nbextensions/toc.css
# !jupyter nbextension enable toc

# enrich using synonim

## load kamus indo

In [457]:
import json
kamus={}
with open("data/dict.json","r") as fi:
    kamus=json.load(fi)

In [458]:
kamus["mengaras"]

{'tag': 'v',
 'sinonim': ['hingga ke',
  'mencapai',
  'mencecah',
  'menjejak',
  'menyentuh',
  'sampai ke']}

# normalize keys of kamus using same preprocessing steps as before

In [464]:
from tqdm import tqdm
from time import sleep
from datetime import datetime
from uuid import uuid4
    
def get_unique_code():
    return datetime.now().strftime('%Y%m-%d%H-%M%S-') + str(uuid4())

kamus_normalized={text_preprocessing(k):{} for k,v in kamus.items()}
# pbar=tqdm(kamus.items())
for k,v in tqdm(kamus.items()):
    try:
#         pbar.set_description("Normalizing kamus on key:{}".format(k),refresh=True)
        new_k=text_preprocessing(k)
        code=get_unique_code()
        kamus[k]["code"]=code
#         kamus_normalized[new_k]=kamus[k]
        kamus_normalized[new_k]["code"]=code
        kamus_normalized[new_k]["sinonim"]=[text_preprocessing(syn) for syn in kamus[k]["sinonim"]]  
    except Exception as ex:
        print("error on key:{}-->{}".format(k,str(ex)))
        continue
#     print("{}:{}".format(k,v))
with open("data/kamus_normalized.json","w") as fo:
    json.dump(kamus_normalized,fo)

100%|██████████| 20139/20139 [00:43<00:00, 458.97it/s]


In [465]:
kamus["mengaras"],kamus_normalized["ara"]

({'tag': 'v',
  'sinonim': ['hingga ke',
   'mencapai',
   'mencecah',
   'menjejak',
   'menyentuh',
   'sampai ke'],
  'code': '202208-1123-4738-6e21679a-276f-4ad3-ac14-faed03749690'},
 {'code': '202208-1123-4818-44fdc79d-6d2d-4f16-9954-594ae9524e14',
  'sinonim': ['bari', 'pawai']})

In [467]:
def check_kamus_by_code(code):
    res={}
    key=""
    res2={}
    key2=""
    for k,v in kamus.items():
        if v["code"]==code:
            res=kamus[k]
            key=k
    for k,v in kamus_normalized.items():
        if v["code"]==code:
            res2=kamus_normalized[k]
            key2=k
    return key,key2,res,res2
check_kamus_by_code("202208-1123-4818-44fdc79d-6d2d-4f16-9954-594ae9524e14")

('perarakan',
 'ara',
 {'tag': 'n',
  'sinonim': ['perbarisan', 'pawai'],
  'code': '202208-1123-4818-44fdc79d-6d2d-4f16-9954-594ae9524e14'},
 {'code': '202208-1123-4818-44fdc79d-6d2d-4f16-9954-594ae9524e14',
  'sinonim': ['bari', 'pawai']})

In [468]:
kamus["siuman"]["code"],kamus_normalized["siuman"]["code"]

('202208-1123-4738-10fb688b-6512-4412-a5c6-532fe301dbf4',
 '202208-1123-4738-10fb688b-6512-4412-a5c6-532fe301dbf4')

In [469]:
len(kamus.keys()),len(kamus_normalized.keys())

(20139, 11486)

# load function to grab synonim from sinonimkata.com in case token not found in offline kamus

In [470]:
def get_synonims_from_sinonimkata_com(q):
    if not q:
        return
    myobj = {}#{'q': 'jurnal'}
    synonim_wrap="rel='nofollow'>[a-zA-Z0-9\s]+</a>"
    url = 'http://www.sinonimkata.com/search.php?q='+q
    x = requests.post(url, json = myobj)
    responds=re.findall(synonim_wrap,x.text)
    token_wrap="rel='nofollow'>(.*)</a>"
    syn_tokens=[]
    for token in responds:
        t=re.findall(token_wrap,token)
        if t:
            syn_tokens.append(t[0])
#     print("get syn for {}-->{}".format(q,",".join(syn_tokens)))
    return syn_tokens
get_synonims_from_sinonimkata_com("jurnal")

['buku harian', 'kronik', 'buletin', 'harian', 'koran', 'surat kabar']

# cleanse kamus_normaized from null synonyms

In [472]:
null_count=0
for k,v in tqdm(kamus_normalized.items(),desc="cleansing synonims from NULL"):
    if "sinonim" in v:
        syns=[]
        for syn in v["sinonim"]:
            if syn is not None:
                syns.append(syn)
            else:
                null_count+=1
        kamus_normalized[k]["sinonim"]=syns
print("total null fund:{}".format(null_count))            
with open("data/kamus_normalized.json","w") as fo:
    json.dump(kamus_normalized,fo)
with open("data/kamus_normalized.json","r") as fi:
    kamus_normalized=json.load(fi)

cleansing synonims from NULL: 100%|██████████| 11486/11486 [00:00<00:00, 198509.92it/s]


total null fund:0


# check kamus vs kamus_normalized

In [473]:
c=0
pbar=tqdm(kamus_normalized.items())
for k,v in pbar:
    try:
        if c==33:
            break
        code=kamus_normalized[k]["code"]
        pbar.set_description("\rchecking key {} code {}".format(k,code))
        key1,key2,dct1,dct2=check_kamus_by_code(code)
        print("{}".format(k))
        print("{}".format(kamus[key1]["sinonim"]))
        print("{}".format(kamus_normalized[key2]["sinonim"]))
        c+=1
    except Exception as ex:
        print("error:{}".format(str(ex)))
        continue
# check_kamus_by_key("tidur")

checking key plagiat code 202208-1123-4801-8e37da74-6b5d-4b0b-ace9-3f9a88c35543:   0%|          | 9/11486 [00:00<04:41, 40.74it/s]  

ara
['perbarisan', 'pawai']
['bari', 'pawai']
daya
['terjerumus', 'terkecoh', 'terkena', 'terkurung', 'terperangkap', 'terpikat', 'terpukau', 'tertangkap', 'tertawan', 'tertipu']
['jerumu', 'kecoh', 'kena', 'kurung', 'perangkap', 'pikat', 'pukau', 'tangkap', 'tawan', 'tipu']
arau
['memutar', 'mengacau', 'mengadon', 'mengaduk', 'mengarun', 'mengebur', 'mengocok']
['putar', 'acau', 'adon', 'aduk', 'arun', 'kebur', 'kocok']
sigi
['obor', 'suluh']
['obor', 'suluh']
anyep
['cahang', 'campah', 'hambar', 'tawar (rasa)', 'adem', 'dingin']
['cahang', 'campah', 'hambar', 'tawar', 'adem', 'dingin']
ceratai
['membahas', 'membicarakan', 'membincangkan', 'menggosipkan', 'menggunjingkan', 'mengomongkan', 'meributkan']
['baha', 'bicara', 'bincang', 'gosip', 'gunj', 'omong', 'ribut']
lelaki
['adam', 'laki-laki', 'lanang', 'maskulin', 'pria']
['adam', 'laki', 'lanang', 'maskulin', 'pria']
maklumat
['memaklumkan', 'memberitahukan', 'memberitakan', 'mengumumkan', 'menyiarkan', 'memublikasikan', 'mewartaka

checking key kerudung code 202208-1123-4754-6754466d-9652-4f5c-8830-3eaf9abe04de:   0%|          | 14/11486 [00:00<04:40, 40.87it/s]    

plagiat
['jiplakan']
['jiplak']
sumpah
['berikrar', 'berjanji', 'mengangkat sumpah']
['ikrar', 'janji', 'angkat sumpah']
jihat
['arah', 'sebelah', 'sisi']
['arah', 'belah', 'sisi']
inkonsistensi
['ketidakseragaman', 'ketidakteraturan', 'ketidaktetapan', 'inkompatibilitas', 'kontradiksi', 'paradoks']
['ketidakseragaman', 'ketidakteraturan', 'ketidaktetapan', 'inkompatibilita', 'kontradiksi', 'paradok']
unta
['berjuntai', 'berenteng', 'beronte']
['junta', 'enteng', 'beront']
arah
['instruktur', 'ketua', 'komandan', 'pemandu', 'pembimbing', 'pemimpin', 'pengatur', 'pengawas', 'pengelola', 'pengendali', 'pengurus', 'penuntun', 'penyelia']
['instruktur', 'ketua', 'komandan', 'pandu', 'bimb', 'pimpin', 'atur', 'awa', 'kelola', 'kendali', 'uru', 'tuntun', 'selia']
arak
['ambalan', 'defile', 'kirab', 'konvoi', 'parade', 'pawai', 'perarakan', 'prosesi']
['ambal', 'defil', 'kirab', 'konvoi', 'parad', 'pawai', 'ara', 'prosesi']
tanggap
['meladeni', 'melayani', 'melihat', 'memahami', 'memandang', 

checking key gabung code 202208-1123-4822-53bcb015-795d-4c31-be7e-7b021399a2de:   0%|          | 24/11486 [00:00<04:38, 41.19it/s]  

kerudung
['jilbab', 'kudung', 'tengkuluk']
['jilbab', 'kudung', 'tengkuluk']
cakar
['perbalahan', 'percekcokan', 'pergaduhan', 'perkelahian', 'pertelingkahan', 'pertengkaran']
['balah', 'cekcok', 'gaduh', 'kelahi', 'telingkah', 'tengkar']
varieta
['jenis', 'macam', 'spesies', 'tipe', 'diversifikasi', 'keberagaman', 'variasi']
['jeni', 'spesi', 'tipe', 'diversifikasi', 'agam', 'variasi']
null
['sedikit demi sedikit']
[]
tap
['membersihkan', 'memilah', 'memilih', 'menampi', 'meneliti', 'mengayak', 'mengidang', 'mengirai', 'mengisai', 'menimbang', 'meninting', 'menyaring', 'menyeleksi', 'menyelerai', 'menyerkai', 'menyortir']
['bersih', 'mem', 'pilih', 'tampi', 'teliti', 'ayak', 'kidang', 'kirai', 'kisai', 'timbang', 'tint', 'nyar', 'seleksi', 'lerai', 'serkai', 'sortir']
biku
['biksu', 'kelim', 'kili', 'pelipit', 'som', 'takik', 'wiru']
['biksu', 'kelim', 'kili', 'lipit', 'som', 'takik', 'wiru']
tuala
['handuk']
['handuk']
wenang
['manasuka', 'sekehendak hati', 'semaunya', 'semau-mau', '

checking key berang code 202208-1123-4756-558d8d78-6a45-48b5-876d-39029b93536d:   0%|          | 33/11486 [00:00<04:38, 41.06it/s]

gabung
['tercampur', 'terhimpun', 'terpadu', 'terkumpul', 'terpumpun']
['campur', 'himpun', 'padu', 'kumpul', 'pumpun']
tarik
['anjur', 'betot', 'cabut', 'eret', 'ganjur', 'hela', 'hunus', 'raih', 'rebut', 'renggut', 'sentak', 'seret']
['anjur', 'betot', 'cabut', 'eret', 'ganjur', 'hela', 'hunu', 'raih', 'rebut', 'renggut', 'sentak', 'seret']
berani
['memberangsangkan', 'mengoja']
['berangsang', 'oja']
bene
['bersih', 'cerah', 'hening', 'jernih', 'menjantang', 'putih', 'resik', 'semerawang', 'suci', 'telus', 'tembus cahaya', 'tembus pandang', 'tembus sinar', 'terang', 'transparan']
['bersih', 'cerah', 'hene', 'jernih', 'jantang', 'putih', 'resik', 'semerawang', 'suci', 'telu', 'tembu cahaya', 'tembu pandang', 'tembu sinar', 'terang', 'transparan']
mafhum
['arif', 'grahita', 'maklum', 'mengerti', 'paham', 'tahu']
['arif', 'grahita', 'maklum', 'erti', 'paham']
berang
['anjing air', 'berang-berang']
['anj air', 'berang']


# get column activity with the longest content

In [474]:
max_l=0
max_idx=0
for idx,row in df_dupak_all.iterrows():
    l=len(str(row.activities))
    if l>max_l:
        max_l=l
        max_idx=idx
#         print("{}){}".format(idx,str(row.activities)))
df_dupak_all.loc[max_idx,["activities"]],df_dupak_all.loc[max_idx,["activity_cleansed"]],max_idx

    

(activities    Mengajar/melatih/membimbing yang berkaitan dengan bidang teknologi informasi berbasis komputer .Melakukan Kegiatan Pembinaan Jabatan Fungsional Pranata Komputer.Lamanya kurang dari 30 jam
 Name: 322, dtype: object,
 activity_cleansed    ajar kait bidang teknolog informasi ba komput giat bina jabat fungsion pranata komput jam
 Name: 322, dtype: object,
 322)

# new column containing enriched activities data

In [475]:
kamus={}
with open("data/dict.json") as fi:
    kamus=json.load(fi)
    
def enrich_activity(txt):
    txt=str(txt).lower()
#     template=txt
#     result=[]
    arr=re.findall("\w+",txt)
    final_result=" ".join(arr)
    template=final_result
#     unique_arr=sorted(set(arr), key=arr.index)
#     result=unique_arr
#     pbar1=tqdm(arr)
    for token in arr:
#         print("enriching token:{}".format(token))
#         pbar1.set_description("enriching token:{}".format(token))
        syns=[]
        #             token_cleansed=text_preprocessing(token)
        try:
            syns=kamus[token]["sinonim"]
        except:
            try:
                syns=kamus_normalized[token]["sinonim"]
            except:
                try:
                    syns=get_synonims_from_sinonimkata_com(token)
                except:
                    print("\rretrying get synonims from sinonimkata.com")
                    time.sleep(randrange(1,2))
                    syns=get_synonims_from_sinonimkata_com(token)
#         syns=[text_preprocessing(s) for s in syns if s]
        for syn in syns:
            if syn is not None:
                a=str(template).replace(token,syn)
                final_result+=". "+a
    return final_result
    
tqdm.pandas()
df_dupak_all["activity_enriched"]=df_dupak_all.activities.progress_apply(enrich_activity)

100%|██████████| 336/336 [01:08<00:00,  4.88it/s]


In [476]:
df_dupak_all.activities.head()

0    Melakukan pemenuhan permintaan dan layanan teknologi informasi
1            Melakukan pengumpulan informasi mengenai data instansi
2            Melakukan pengadministrasian kegiatan tata kelola data
3                Melakukan pencatatan permasalahan pengelolaan data
4                     Melakukan perawatan arsitektur teknologi data
Name: activities, dtype: object

In [477]:
df_dupak_all.activity_enriched.head()

0                                                                                                                                  melakukan pemenuhan permintaan dan layanan teknologi informasi. melaksanakan pemenuhan permintaan dan layanan teknologi informasi. melancarkan pemenuhan permintaan dan layanan teknologi informasi. melangsungkan pemenuhan permintaan dan layanan teknologi informasi. melantaskan pemenuhan permintaan dan layanan teknologi informasi. melayani pemenuhan permintaan dan layanan teknologi informasi. mengadakan pemenuhan permintaan dan layanan teknologi informasi. menjalankan pemenuhan permintaan dan layanan teknologi informasi. menunaikan pemenuhan permintaan dan layanan teknologi informasi. menyelenggarakan pemenuhan permintaan dan layanan teknologi informasi. membuat pemenuhan permintaan dan layanan teknologi informasi. mengerjakan pemenuhan permintaan dan layanan teknologi informasi. menggarap pemenuhan permintaan dan layanan teknologi informasi. meluluskan pemen

In [398]:
df_dupak_all.loc[281,["activity_enriched"]]

activity_enriched    dalam bentuk buku majalah ilmiah internasional yang diterbitkan internasional yang terindek. analitis bentuk buku majalah ilmiah internasional yang diterbitkan internasional yang terindek. batin (hati) bentuk buku majalah ilmiah internasional yang diterbitkan internasional yang terindek. berarti (maksud) bentuk buku majalah ilmiah internasional yang diterbitkan internasional yang terindek. berbobot bentuk buku majalah ilmiah internasional yang diterbitkan internasional yang terindek. berisi bentuk buku majalah ilmiah internasional yang diterbitkan internasional yang terindek. berkualitas bentuk buku majalah ilmiah internasional yang diterbitkan internasional yang terindek. bermakna bentuk buku majalah ilmiah internasional yang diterbitkan internasional yang terindek. bermutu bentuk buku majalah ilmiah internasional yang diterbitkan internasional yang terindek. bernas bentuk buku majalah ilmiah internasional yang diterbitkan internasional yang terindek. jeluk bentuk

# cleanse enriched data

In [478]:
tqdm.pandas()
df_dupak_all["activity_enriched_cleansed"]=df_dupak_all.activity_enriched.progress_apply(text_preprocessing)

100%|██████████| 336/336 [07:09<00:00,  1.28s/it]


In [479]:
df_dupak_all.loc[281,["activity_cleansed","activity_enriched_cleansed"]]

activity_cleansed                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

# save dataframe with activity enriched

In [480]:
df_dupak_all.to_csv("data/dupak_all.csv",sep=";")

# retrain tfidf and Knn using enriched data

In [481]:
tfidf_model_fp_enriched="model/tfdif_vectorizer_enriched.pkl"
knn_model_path_enriched="model/knn_enriched.pkl"
knn_index_path_enriched="model/knn_idx_enriched.pkl"

activity_enriched=df_dupak_all.activity_enriched_cleansed.to_list()
tfidf_vectorizer_enriched=TfidfVectorizer(use_idf=True,ngram_range=(1,2))
fitted_tfidf_vectorizer_enriched = tfidf_vectorizer_enriched.fit(activity_enriched)
with open(tfidf_model_fp_enriched, "wb") as fo:
        pickle.dump(fitted_tfidf_vectorizer_enriched, fo)
print("SAVED tfidf vectorizer to ", tfidf_model_fp_enriched)



def vectorize_tfidf_enriched(txt):
    densematrix=fitted_tfidf_vectorizer_enriched.transform([txt])
    skillvecs=densematrix.toarray().tolist()
    vector=np.array(skillvecs[0]).astype('float32').tolist()
#     vector=np.expand_dims(vector,0).tolist()
    return vector
df_dupak_all["tfidf_vec_enriched"]=df_dupak_all.activity_enriched_cleansed.apply(vectorize_tfidf_enriched)
df_dupak_all.to_csv("data/dupak_all.csv",sep=";")


df_dupak_all.reset_index(inplace=True, drop=True)



knn_enriched = KNeighborsClassifier()
knn_index_enriched={}
vecs_enriched=df_dupak_all.tfidf_vec_enriched.to_list()
idxs_enriched=df_dupak_all.index.to_list()
activities=df_dupak_all.activities.to_list()
for i,val in enumerate(activities):
    knn_index_enriched[i]=val
knn_enriched.fit(vecs_enriched, idxs_enriched)
with open(knn_model_path_enriched,"wb") as fi:
  pickle.dump(knn_enriched,fi)
  print("train KNN DONE. Model saved in ",knn_model_path_enriched)
with open(knn_index_path_enriched,"wb") as fi:
  pickle.dump(knn_index_enriched,fi)
  print("Index saved in ",knn_index_path_enriched)





SAVED tfidf vectorizer to  model/tfdif_vectorizer_enriched.pkl
train KNN DONE. Model saved in  model/knn_enriched.pkl
Index saved in  model/knn_idx_enriched.pkl


In [407]:
# from nltk.tag import CRFTagger
# ct = CRFTagger()
# ct.set_model_file('model/all_indo_man_tag_corpus_model.crf.tagger')
# hasil = ct.tag_sents([['Saya','bekerja','di','Bandung']])
# s='penuh minta layan teknolog informasi'
# s2='Melakukan pemenuhan permintaan dan layanan teknologi informasi'
# ct.tag_sents([s.split()]),ct.tag_sents([s2.split()])

# compare tfidf model 1 and 2

In [482]:
%%time
merge_result={"vanilla":{},"enriched":{}}
q="jurnal internasional"
q_cleansed=text_preprocessing(q)
q_vector=vectorize_tfidf(q_cleansed)
(distances, indices) = knn.kneighbors([q_vector],n_neighbors=5)
indices = indices.tolist()
res=[knn_index[x] for x in indices[0]]
print("Q:{}".format(q))
for i,x in enumerate(res):
    print("{}){}-->idx:{} distance:{} AK:{}".format(i,x,indices[0][i],distances[0][i],df_dupak_all.loc[indices[0][i],["ak"]][0]))
    merge_result["vanilla"][i]={"index":indices[0][i],"distance":distances[0][i],"activity":x,"ak":df_dupak_all.loc[indices[0][i],["ak"]][0]}


Q:jurnal internasional
0)Tingkat Internasional-->idx:327 distance:1.1151716708765296 AK:35% AK kenaikan
pangkat
1)dalam bentuk buku/majalah ilmiah internasional yang diterbitkan internasional yang terindek-->idx:281 distance:1.1447162451190245 AK:20
2)dalam bentuk buku/majalah ilmiah internasional yang diterbitkan nasional-->idx:282 distance:1.2586365510044788 AK:12.5
3)Menjadi Editor Karya Tulis Ilmiah, Jurnal, Konferensi, Forum Ilmiah
dan Lain-Lain di Bidang Teknologi Informasi.Lamanya kurang dari 30 jam-->idx:318 distance:1.2927334438241374 AK:0.5
4)Menjadi Reviewer Karya Tulis Ilmiah, Jurnal, Konferensi, Forum Ilmiah
dan Lain-Lain di Bidang Teknologi Informasi.Lamanya kurang dari 30 jam-->idx:317 distance:1.2927334438241374 AK:0.5
CPU times: user 263 ms, sys: 8.16 ms, total: 271 ms
Wall time: 43.7 ms


In [483]:
%%time
q="jurnal internasional"
q_cleansed=text_preprocessing(q)
q_vector=vectorize_tfidf_enriched(q_cleansed)
(distances, indices) = knn_enriched.kneighbors([q_vector],n_neighbors=5)
indices = indices.tolist()
res=[knn_index_enriched[x] for x in indices[0]]
print("Q:{}".format(q))
for i,x in enumerate(res):
    print("{}){}-->idx:{} distance:{} AK:{}".format(i,x,indices[0][i],distances[0][i],df_dupak_all.loc[indices[0][i],["ak"]][0]))
    merge_result["enriched"][i]={"index":indices[0][i],"distance":distances[0][i],"activity":x,"ak":df_dupak_all.loc[indices[0][i],["ak"]][0]}


Q:jurnal internasional
0)Tingkat Internasional-->idx:327 distance:1.0404298453804781 AK:35% AK kenaikan
pangkat
1)dalam bentuk buku/majalah ilmiah internasional yang diterbitkan internasional yang terindek-->idx:281 distance:1.1290180289677365 AK:20
2)dalam bentuk buku/majalah ilmiah internasional yang diterbitkan nasional-->idx:282 distance:1.2506157077020703 AK:12.5
3)dalam bentuk buku/majalah ilmiah internasional yang diterbitkan dan diakui oleh organisasi profesi dan Instansi Pembina-->idx:283 distance:1.2843768381700806 AK:6
4)Menjadi Reviewer Karya Tulis Ilmiah, Jurnal, Konferensi, Forum Ilmiah
dan Lain-Lain di Bidang Teknologi Informasi.Lamanya kurang dari 30 jam-->idx:317 distance:1.3155125513236028 AK:0.5
CPU times: user 302 ms, sys: 23.9 ms, total: 326 ms
Wall time: 90 ms


In [484]:
for i in range(5):
    lbl=""
    if merge_result["vanilla"][i]["distance"] < merge_result["enriched"][i]["distance"]:
        idx=merge_result["vanilla"][i]["index"]
        lbl="vanilla"
        ak=merge_result["vanilla"][i]["ak"]
    else:
        idx=merge_result["enriched"][i]["index"]
        lbl="enriched"
        ak=merge_result["vanilla"][i]["ak"]
    if lbl=="vanilla":
        print("{},{}x{}.<--{}".format(knn_index_enriched[idx],ak,merge_result["enriched"][i]["ak"],lbl))
    else:
        print("{},{}x{}.<--{}".format(knn_index_enriched[idx],ak,merge_result["vanilla"][i]["ak"],lbl))
  
    

Tingkat Internasional,35% AK kenaikan
pangkatx35% AK kenaikan
pangkat.<--enriched
dalam bentuk buku/majalah ilmiah internasional yang diterbitkan internasional yang terindek,20x20.<--enriched
dalam bentuk buku/majalah ilmiah internasional yang diterbitkan nasional,12.5x12.5.<--enriched
dalam bentuk buku/majalah ilmiah internasional yang diterbitkan dan diakui oleh organisasi profesi dan Instansi Pembina,0.5x0.5.<--enriched
Menjadi Reviewer Karya Tulis Ilmiah, Jurnal, Konferensi, Forum Ilmiah
dan Lain-Lain di Bidang Teknologi Informasi.Lamanya kurang dari 30 jam,0.5x0.5.<--vanilla


# long story short for tfidf (combi of vanilla and enriched model)

In [486]:
q="instalasi perangkat server untuk pengadaan big data"
merge_result={"vanilla":{},"enriched":{}}
q_cleansed=text_preprocessing(q)
q_vector=vectorize_tfidf(q_cleansed)
(distances, indices) = knn.kneighbors([q_vector],n_neighbors=5)
indices = indices.tolist()
res=[knn_index[x] for x in indices[0]]
print("Q:{}".format(q))
for i,x in enumerate(res):
    print("{}){}-->idx:{} distance:{} AK:{}".format(i,x,indices[0][i],distances[0][i],df_dupak_all.loc[indices[0][i],["ak"]][0]))
    merge_result["vanilla"][i]={"index":indices[0][i],"distance":distances[0][i],"activity":x,"ak":df_dupak_all.loc[indices[0][i],["ak"]][0]}
print("=====================================================")
q_cleansed=text_preprocessing(q)
q_vector=vectorize_tfidf_enriched(q_cleansed)
(distances, indices) = knn_enriched.kneighbors([q_vector],n_neighbors=5)
indices = indices.tolist()
res=[knn_index_enriched[x] for x in indices[0]]
# print("Q:{}".format(q))
for i,x in enumerate(res):
    print("{}){}-->idx:{} distance:{} AK:{}".format(i,x,indices[0][i],distances[0][i],df_dupak_all.loc[indices[0][i],["ak"]][0]))
    merge_result["enriched"][i]={"index":indices[0][i],"distance":distances[0][i],"activity":x,"ak":df_dupak_all.loc[indices[0][i],["ak"]][0]}
print("=====================================================")
for i in range(5):
    lbl=""
    if merge_result["vanilla"][i]["distance"] < merge_result["enriched"][i]["distance"]:
        idx=merge_result["vanilla"][i]["index"]
        lbl="vanilla"
        ak=merge_result["vanilla"][i]["ak"]
    else:
        idx=merge_result["enriched"][i]["index"]
        lbl="enriched"
        ak=merge_result["vanilla"][i]["ak"]
    if lbl=="vanilla":
        print("{}) {},{}x{}.<--{}".format(i,knn_index_enriched[idx],ak,merge_result["enriched"][i]["ak"],lbl))
    else:
        print("{}) {},{}x{}.<--{}".format(i,knn_index_enriched[idx],ak,merge_result["vanilla"][i]["ak"],lbl))

Q:instalasi perangkat server untuk pengadaan big data
0)Melakukan instalasi/updating DBMS-->idx:18 distance:1.122351297489197 AK:0.055
1)Melakukan instalasi dan konfigurasi DBMS-->idx:168 distance:1.1923660095245319 AK:0.12
2)Melakukan instalasi/upgrade dan konfigurasi sistem operasi/aplikasi-->idx:259 distance:1.215561755630454 AK:0.11
3)Melakukan pemasangan perangkat fisik TI-->idx:46 distance:1.2288131520676568 AK:0.165
4)Melakukan pengujian perangkat TI End User-->idx:42 distance:1.2410606488619975 AK:0.225
0)Melakukan instalasi/upgrade sistem operasi komputer/perangkat lunak pada infrastruktur TI-->idx:49 distance:1.191000390099888 AK:0.042
1)Melakukan instalasi dan konfigurasi DBMS-->idx:168 distance:1.1968963876206786 AK:0.12
2)Melakukan instalasi/updating DBMS-->idx:18 distance:1.2149537301311069 AK:0.055
3)Melakukan instalasi/upgrade dan konfigurasi sistem operasi/aplikasi-->idx:259 distance:1.2576386547513811 AK:0.11
4)Melakukan pengujian perangkat TI End User-->idx:42 distan

In [418]:
# df_dupak_all.loc[264,["activity_cleansed","activity_enriched_cleansed"]]

# try google embedding
https://indobenchmark.github.io/tutorials/pytorch/deep%20learning/nlp/2020/10/18/basic-pytorch-id.html

In [487]:
df_dupak_all.activities.head(2)

0    Melakukan pemenuhan permintaan dan layanan teknologi informasi
1            Melakukan pengumpulan informasi mengenai data instansi
Name: activities, dtype: object

In [ ]:
# !TMPDIR=/home/pusintek/tmp pip install --cache-dir=$TMPDIR --build=$TMPDIR tensorflow-gpu

# load indobert model

In [488]:
# import sentencepiece
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification

bert_tokenizer = BertTokenizer.from_pretrained("model")
bert_config = BertConfig.from_pretrained("model")
bert_model = BertForSequenceClassification.from_pretrained("model", config=bert_config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [489]:
# x = torch.LongTensor(bert_tokenizer.encode('aku adalah anak [MASK]')).view(1,-1)
# print(x)
# print(bert_model(x)[0].detach().numpy()[0])
# print(bert_model(x)[0].sum())
from scipy import spatial

def get_cosine_similarity(a,b):
    simscore = 1 - spatial.distance.cosine(a, b)
    return simscore
def vectorize_by_indobert(txt):
    txt=txt+ " [MASK]"
    x = torch.LongTensor(bert_tokenizer.encode(txt)).view(1,-1)
    return bert_model(x)[0].detach().numpy()[0].tolist()
# v1=vectorize_by_indobert('menyusun jurnal ilmiah')
# v2=vectorize_by_indobert('main bola di lapangan')
# v3=vectorize_by_indobert('instalasi database')
# print(get_cosine_similarity(v1,v2))
# print(get_cosine_similarity(v1,v3))
# print(get_cosine_similarity(v2,v3))

# vectorize the activities using to indobert vector

In [490]:
tqdm.pandas()
df_dupak_all["indobert_vec"]=df_dupak_all.activities.progress_apply(vectorize_by_indobert)

100%|██████████| 336/336 [00:17<00:00, 19.21it/s]


# save df to csv

In [491]:
df_dupak_all.to_csv("data/dupak_all.csv",sep=";")

# train knn on indobert vecs

In [492]:
knn_model_path_indobert="model/knn_indobert.pkl"
knn_index_path_indobert="model/knn_index_indobert.pkl"
knn_indobert = KNeighborsClassifier()
knn_index_indobert={}
vecs_indobert=df_dupak_all.indobert_vec.to_list()
idxs_indobert=df_dupak_all.index.to_list()
activities=df_dupak_all.activities.to_list()
for i,val in enumerate(activities):
    knn_index_indobert[i]=val
knn_indobert.fit(vecs_indobert, idxs_indobert)
with open(knn_model_path_indobert,"wb") as fi:
  pickle.dump(knn_indobert,fi)
  print("train KNN DONE. Model saved in ",knn_model_path_indobert)
with open(knn_index_path_indobert,"wb") as fi:
  pickle.dump(knn_index_indobert,fi)
  print("Index saved in ",knn_index_path_indobert)

train KNN DONE. Model saved in  model/knn_indobert.pkl
Index saved in  model/knn_index_indobert.pkl


# try quey knn + indobert

In [494]:
q="instalasi perangkat server untuk pengadaan big data"
q_vector=vectorize_by_indobert(q)
(distances, indices) = knn_indobert.kneighbors([q_vector],n_neighbors=5)
indices = indices.tolist()
res=[knn_index_indobert[x] for x in indices[0]]
# print("Q:{}".format(q))
for i,x in enumerate(res):
    print("{}){}-->idx:{} distance:{} AK:{}".format(i,x,indices[0][i],distances[0][i],df_dupak_all.loc[indices[0][i],["ak"]][0]))



0)Melakukan implementasi data model-->idx:148 distance:0.08241962855812317 AK:0.22
1)Menyusun komponen Enterprise Architecture (EA) saat ini-->idx:83 distance:0.092896136861438 AK:1.32
2)Melakukan implementasi teknologi data-->idx:22 distance:0.10132851006563152 AK:0.11
3)Menyusun pengembangan komponen Enterprise Architecture (EA ) masa yang akan datang-->idx:84 distance:0.10254615603709273 AK:4.84
4)Melakukan perancangan data model-->idx:147 distance:0.10275202989941994 AK:0.66


# use BERT english and translator :)

In [191]:
# !pip uninstall -y googletrans
# !pip uninstall -y googletrans==3.1.0a0
# !pip install googletrans==4.0.0-rc1
# !pip install translators --upgrade

In [417]:
# from googletrans import Translator, constants
# # from pprint import pprint
# from google_trans_new2 import google_trans_new
import translators as ts

In [197]:

# import sys
# sys.path.insert(1,"/home/pusintek/project/andri/dupakq/google_trans_new")
# sys.path.insert(2,"/home/pusintek/project/andri/dupakq/constant")
# from google_trans_new2 import google_trans_new
# # from google_trans_new  import google_translator
# translator = google_translator()  
# translate_text = translator.translate('Hola mundo!', lang_src='es', lang_tgt='en')  
# print(translate_text)
# ts._google.language_map

In [495]:
phrase = 'menyusun jurnal ilmiah internasional.'
ts.google(phrase, from_language='id', to_language='en',sleep_seconds=randrange(1,2))

'Develop international scientific journals.'

In [496]:
def translate_to_en(txt):
    en=""
    try:
        en= ts.google(txt, from_language='id', to_language='en',sleep_seconds=random.uniform(0.5, 1.9))
    except:
        try:
            print("\rretry using google translate",end="")
            en= ts.google(txt, from_language='id', to_language='en',sleep_seconds=random.uniform(0.5, 1.9))
        except:
            print("cannot translate:{}".format(txt))
    return en    
    
    
# a="ayo pergi ke masjid"
# b="di sana ada mangkuk air zam-zam"
# c="segar dan berkah"
# for id in [a,b,c]:
#     t0=datetime.now()
#     en=translate_to_en(id)
#     print("{}-->{}".format(id,en))
#     t1=datetime.now()
#     diff=t1-t0
#     print("sleep={}".format(diff.seconds))
    

# new column contaning english translation of acivities

In [497]:
tqdm.pandas()
df_dupak_all["activity_en"]=df_dupak_all.activities.progress_apply(translate_to_en)

 45%|████▍     | 151/336 [05:04<06:13,  2.02s/it]

retry using google translate

 89%|████████▉ | 299/336 [10:25<01:52,  3.05s/it]

retry using google translate

 89%|████████▉ | 300/336 [11:05<08:29, 14.15s/it]

cannot translate:Seminar/lokakarya/konferensi/simposium/studi banding lapangan.dalam bentuk makalah
retry using google translate

100%|██████████| 336/336 [12:29<00:00,  2.23s/it]


In [501]:
x="Seminar/lokakarya/konferensi/simposium/studi banding lapangan.dalam bentuk makalah"
df_dupak_all[df_dupak_all.activities.str.contains(x)][["activities","activity_code"]]

,activities,activity_code
298,Seminar/lokakarya/konferensi/simposium/studi banding lapangan.dalam bentuk makalah,Semua Jenjang_IV.E.2


In [505]:
df_dupak_all.at[298,"activities"]='Seminar atau lokakarya atau konferensi atau simposium atau studi banding lapangan dalam bentuk makalah'

In [507]:
df_dupak_all.at[298,"activities"]

'Seminar atau lokakarya atau konferensi atau simposium atau studi banding lapangan dalam bentuk makalah'

In [508]:
df_dupak_all.at[298,"activity_en"]=translate_to_en(df_dupak_all.at[298,"activities"])

In [509]:
df_dupak_all.at[298,"activity_en"]

'Seminars or workshops or conferences or symposiums or field comparative studies in the form of papers'

# save dataframe to csv

In [510]:
df_dupak_all.to_csv("data/dupak_all.csv",sep=";")

In [511]:
len(df_dupak_all.activity_code.to_list()),len(df_dupak_all.activity_code.unique())

(336, 336)

In [512]:
df_dupak_all["activity_en"].head()

0    Fulfillment of Demand and Information Technology Services
1                        Conduct information about agency data
2                     Administering data management activities
3                        Record the problem of data management
4            Conduct data architecture data on data technology
Name: activity_en, dtype: object

# get english bert model

In [513]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
# tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
# model = model_class.from_pretrained(pretrained_weights)
tokenizer = tokenizer_class.from_pretrained("model/en_bert_model")
model = model_class.from_pretrained("model/en_bert_model")

In [220]:
# model.save_pretrained("model/en_bert_model")

In [514]:
# tokenizer.save_pretrained("model/en_bert_model")

In [515]:
tqdm.pandas()
tokenized = df_dupak_all.activity_en.progress_apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

100%|██████████| 336/336 [00:00<00:00, 1862.09it/s]


In [516]:
tokenized.head(2)

0    [101, 29362, 1997, 5157, 1998, 2592, 2974, 2578, 102]
1                 [101, 6204, 2592, 2055, 4034, 2951, 102]
Name: activity_en, dtype: object

In [517]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
padded

array([[  101, 29362,  1997, ...,     0,     0,     0],
       [  101,  6204,  2592, ...,     0,     0,     0],
       [  101, 28965,  2951, ...,     0,     0,     0],
       ...,
       [  101,  3460,  1012, ...,     0,     0,     0],
       [  101,  6594,  1998, ...,     0,     0,     0],
       [  101,  3447,  1997, ...,     0,     0,     0]])

In [536]:
max_len

33

In [518]:
np.array(padded).shape

(336, 33)

In [519]:
padded[0]

array([  101, 29362,  1997,  5157,  1998,  2592,  2974,  2578,   102,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0])

In [520]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(336, 33)

In [521]:
attention_mask[:2]

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [522]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [523]:
len([x for x in last_hidden_states])

1

In [524]:
type(last_hidden_states)

transformers.modeling_outputs.BaseModelOutput

In [525]:
last_hidden_states[0]

tensor([[[-0.2384, -0.0794, -0.3115,  ..., -0.4130,  0.2669,  0.1059],
         [ 0.0864,  0.4610, -0.1764,  ..., -0.3894,  0.3040, -0.4283],
         [-0.3725,  0.1786, -0.2746,  ..., -0.6169,  0.1472, -0.3323],
         ...,
         [ 0.2961, -0.0037,  0.2913,  ..., -0.2483, -0.2466, -0.0198],
         [ 0.3678,  0.0308,  0.1459,  ..., -0.0993, -0.3527, -0.1773],
         [ 0.3941,  0.0149,  0.1252,  ..., -0.0574, -0.3984, -0.2545]],

        [[-0.1246,  0.1522, -0.4071,  ..., -0.4243,  0.1112,  0.4241],
         [ 0.6646,  0.7761, -0.0115,  ..., -0.3397,  0.3953,  0.2802],
         [-0.3118,  0.2906,  0.0245,  ..., -0.6156, -0.2054,  0.2749],
         ...,
         [ 0.2567, -0.0410, -0.0588,  ...,  0.0058, -0.2783,  0.0183],
         [ 0.2828, -0.0186, -0.0459,  ...,  0.0213, -0.2813,  0.0206],
         [ 0.3133, -0.0573, -0.0398,  ...,  0.0358, -0.3164,  0.0255]],

        [[-0.2624,  0.1423, -0.2882,  ..., -0.1529,  0.0636,  0.3190],
         [ 0.1426,  0.5666, -0.1133,  ..., -0

In [539]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
# !TMPDIR=/home/pusintek/tmp pip install --cache-dir=$TMPDIR --build=$TMPDIR torchvision

In [420]:
# !TMPDIR=/home/pusintek/tmp pip install --cache-dir=$TMPDIR --build=$TMPDIR sentencepiece

In [529]:
last_hidden_states[0]

tensor([[[-0.2384, -0.0794, -0.3115,  ..., -0.4130,  0.2669,  0.1059],
         [ 0.0864,  0.4610, -0.1764,  ..., -0.3894,  0.3040, -0.4283],
         [-0.3725,  0.1786, -0.2746,  ..., -0.6169,  0.1472, -0.3323],
         ...,
         [ 0.2961, -0.0037,  0.2913,  ..., -0.2483, -0.2466, -0.0198],
         [ 0.3678,  0.0308,  0.1459,  ..., -0.0993, -0.3527, -0.1773],
         [ 0.3941,  0.0149,  0.1252,  ..., -0.0574, -0.3984, -0.2545]],

        [[-0.1246,  0.1522, -0.4071,  ..., -0.4243,  0.1112,  0.4241],
         [ 0.6646,  0.7761, -0.0115,  ..., -0.3397,  0.3953,  0.2802],
         [-0.3118,  0.2906,  0.0245,  ..., -0.6156, -0.2054,  0.2749],
         ...,
         [ 0.2567, -0.0410, -0.0588,  ...,  0.0058, -0.2783,  0.0183],
         [ 0.2828, -0.0186, -0.0459,  ...,  0.0213, -0.2813,  0.0206],
         [ 0.3133, -0.0573, -0.0398,  ...,  0.0358, -0.3164,  0.0255]],

        [[-0.2624,  0.1423, -0.2882,  ..., -0.1529,  0.0636,  0.3190],
         [ 0.1426,  0.5666, -0.1133,  ..., -0

In [532]:
features.shape,df_dupak_all.shape

((336, 768), (336, 17))

In [564]:
max_txt_len=33
def vectorize_by_distilbert(txt_en):
    vec=[]
#     txt_en=translate_to_en(txt_id)
    if not txt_en:
        return []
    else:
        bert_tokens=tokenizer.encode(txt_en, add_special_tokens=True)
        padded = np.array([bert_tokens + [0]*(max_txt_len-len(bert_tokens))])
        attention_mask = np.where(padded != 0, 1, 0)
        input_ids = torch.tensor(padded)
        attention_mask = torch.tensor(attention_mask)
        with torch.no_grad():
            last_hidden_states = model(input_ids, attention_mask=attention_mask)
        vec = last_hidden_states[0][:,0,:].numpy()
    return vec[0].tolist()

# v=vectorize_by_diltilert("compile research on big data")
# len(v),v

In [559]:
# df_dupak_all.drop("distilbert_vec",axis=1,inplace=True)

In [565]:
df_dupak_all["distilbert_vec"]=df_dupak_all.activity_en.progress_apply(vectorize_by_distilbert)

100%|██████████| 336/336 [00:12<00:00, 27.12it/s]


# save data to csv

In [566]:
df_dupak_all.to_csv("data/dupak_all.csv",sep=";")

# get labels from activity code

In [567]:
labels = df_dupak_all.activity_code.to_list()
labels[:5]

['Mahir_I.A.1', 'Mahir_I.B.1', 'Mahir_I.B.2', 'Mahir_I.B.3', 'Mahir_I.B.4']

# train KNN on distilbert_vec

In [568]:
knn_model_path_distilbert="model/knn_distilbert.pkl"
knn_index_path_distilbert="model/knn_index_distilbert.pkl"
knn_distilbert = KNeighborsClassifier()
knn_index_distilbert={}
vecs_distilbert=df_dupak_all.distilbert_vec.to_list()
idxs_distilbert=df_dupak_all.index.to_list()
activities=df_dupak_all.activities.to_list()
for i,val in enumerate(activities):
    knn_index_distilbert[i]=val
knn_distilbert.fit(vecs_distilbert, idxs_distilbert)
with open(knn_model_path_distilbert,"wb") as fi:
  pickle.dump(knn_distilbert,fi)
  print("train KNN DONE. Model saved in ",knn_model_path_distilbert)
with open(knn_index_path_distilbert,"wb") as fi:
  pickle.dump(knn_index_distilbert,fi)
  print("Index saved in ",knn_index_path_distilbert)

train KNN DONE. Model saved in  model/knn_distilbert.pkl
Index saved in  model/knn_index_distilbert.pkl


# try query on distilbert vec

In [578]:
q="membuat kajian big data dan tableau cluster"
q_en=translate_to_en(q)
q_vector=vectorize_by_distilbert(q_en)
(distances, indices) = knn_distilbert.kneighbors([q_vector],n_neighbors=5)
indices = indices.tolist()
res=[knn_index_distilbert[x] for x in indices[0]]
# print("Q:{}".format(q))
print("Q:{}/{}".format(q,q_en))
for i,x in enumerate(res):
    print("{}){}-->idx:{} distance:{} AK:{}".format(i,x,indices[0][i],\
                                                    distances[0][i],df_dupak_all.loc[indices[0][i],\
                                                                                     ["ak"]][0]))
    print("{}) {}".format(i, df_dupak_all.loc[indices[0][i],["activity_en"]][0]))

Q:membuat kajian big data dan tableau cluster/Create a Big Data and Tableau Cluster study
0)Membuat objek multimedia kompleks dengan piranti lunak-->idx:276 distance:2.625206087620338 AK:0.165
0) Create a complex multimedia object with software
1)Membuat program multimedia kompleks-->idx:278 distance:2.7016133595419967 AK:0.6
1) Create a complex multimedia program
2)Melakukan peningkatan kinerja database-->idx:172 distance:2.828334303239645 AK:0.11
2) Make a database performance improvement
3)Menyusun rancangan uji coba sistem jaringan kompleks-->idx:213 distance:2.8662661629596635 AK:0.11
3) Compile a complex network system trial design
4)Menyusun skenario uji coba sistem informasi-->idx:253 distance:2.879443041713702 AK:0.14
4) Compile a trial scenario of information systems


In [575]:
df_dupak_all.loc[indices[0][i],["activity_en"]][0]

'Compile a network security policy formulation'

# summary: use tfidf wkwkwk!

# build the API